In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/My Drive')
!pwd

/content/drive/My Drive


In [ ]:
#imports
import glob
import os
import numpy as np
import random
import time
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import math
 
import cv2
from PIL import Image
import torch
torch.cuda.empty_cache()
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data
from torchvision import models
from torchvision.transforms import ToTensor, Normalize
import torch.optim as optim
import logging
from ipywidgets import IntProgress
import pandas as pd
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, utils, transforms
import re
from torch.nn import ReplicationPad3d

In [ ]:
# global variable
device = torch.device("cuda:0")



In [ ]:

class PoseModel(nn.Module):

    def __init__(self):
        super(PoseModel, self).__init__()
        model1 = models.resnet50(pretrained=True)
        model2 = models.resnet50(pretrained=True)
        
        self.enc1 = nn.Sequential(*list(model1.children())[:-2])
        self.enc2 = nn.Sequential(*list(model2.children())[:-2])       
        
        self.conv1 = nn.Conv2d(4096, 2048, kernel_size=(3,3), stride=(2,2), padding=(1,1))
        self.bn1 = nn.BatchNorm2d(2048)
        
        self.conv2 = nn.Conv2d(2048, 1024, kernel_size=(3,3), stride=(2,2), padding=(1,1))  
        self.bn2 = nn.BatchNorm2d(1024)

        self.conv3 = nn.Conv2d(1024, 512, kernel_size=(3,3), stride=(2,2), padding=(1,1))  
        self.bn3 = nn.BatchNorm2d(512)

        self.conv4 = nn.Conv2d(512, 256, kernel_size=(3,3), stride=(2,2), padding=(1,1))  
        self.bn4 = nn.BatchNorm2d(256)

        self.fc1 = nn.Linear(512, 128)
        self.out= nn.Linear(128,5)   
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, s): 
        
        x1 = torch.clone(s)
        x2 = torch.flip(s,[1])
        x = torch.cat([x1,x2],0)
        
        feature1 = self.enc1(x[:,0])
        feature2 = self.enc2(x[:,1])
        
        feature = torch.cat([feature1,feature2],1)


        x = F.relu(self.bn1(self.conv1(feature)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = torch.reshape(x, (x.size(0),-1))

        x = self.fc1(x)
        x = self.dropout(x)
        
        x = self.out(x)
        out = x.view(s.size(0),2,5)     
        
        return out


net = PoseModel()


dummy_input = torch.rand((2, 2, 3, 480, 640))
output = net(dummy_input)

print(net)
print(output.shape)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth



PoseModel(
  (enc1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d(64, 

In [ ]:
img1 = 'Copy of data1608_90.png'  # raw image 1
img2 = 'Copy of data1608_91.png'  # raw image 2
images = [cv2.cvtColor(cv2.imread(img1), cv2.COLOR_BGR2RGB),cv2.cvtColor(cv2.imread(img2), cv2.COLOR_BGR2RGB)]

data = np.array(images)
        
data = torch.from_numpy(data)

test_model = PoseModel()

test_model.load_state_dict(torch.load('best_model.torch', map_location='cpu')) # test_model.load_state_dict(torch.load('best_model.torch', map_location='cuda'))

test_model.eval()

print(data.shape)


x = data.transpose_(2,3).transpose_(1,2).type(torch.FloatTensor) # data.transpose_(2,3).transpose_(1,2).type(torch.cuda.FloatTensor) 

x = torch.unsqueeze(x, 0)

print(x.shape)

y_pred = test_model(x)


torch.Size([2, 480, 640, 3])
torch.Size([1, 2, 3, 480, 640])


In [ ]:
print(y_pred)

tensor([[[  9.9115, -13.6229, -11.9482,   1.5575,  -2.3205],
         [  9.8581, -13.5162, -11.9875,   1.5439,  -2.2894]]],
       grad_fn=<ViewBackward>)
